In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
matplotlib.rc('font', **{'family':'serif', 'serif':['Computer Modern']})

In [ ]:
load("spritz_equations.sage")

In [ ]:
def simple(N):
    KB = compute_complexity(
        N, 0,
        generate_spritz_variables,
        get_spritz_boundary_conditions,
        generate_spritz_simple_equations,
        return_kb=True
    )
    ff = falling_factorial
    bruteforce = [ff(N,x) for x in range(N)]
    # print bruteforce
    backtrack = [KB[cvar(1,x)] for x in range(N)]
    # print backtrack

    complexity = [a*b for a,b in zip(bruteforce,backtrack)]
    # print complexity
    return complexity

def change_order(N):
    KB = compute_complexity(
        N, 0,
        generate_spritz_variables,
        get_spritz_boundary_conditions,
        generate_spritz_change_order_equations,
        return_kb=True
    )
    ff = falling_factorial
    bruteforce = [ff(N,x) for x in range(N)]
    # print bruteforce
    backtrack = [KB[cvar(1,x)] for x in range(N)]
    # print backtrack

    complexity = [a*b for a,b in zip(bruteforce,backtrack)]
    # print complexity
    return complexity

In [ ]:
def generate_spritz_threshold_equations(N, x, KB, threshold):
    eqs = []
    for i in range(1, 5):
        eqs.append(
            cvar(i,x) == x/N * cvar(i+1,x) + (1 - x/N) * (N-x) * (1 + KB[cvar(i+1,x+1)])
        )
    
    if x < threshold:
        eq5 = c5(x) == ((1 - x/N)^2 * (1 + KB[c1(x+1)]) + x/N * 1/N * c1(x))
        eqs.append(eq5)
        return (eqs, [cvar(i,x) for i in range(1, 6)])
    
    eq5 = c5(x) == x/N * ((1 - x/N)^2 * (1 + KB[c1(x+1)]) + x/N * 1/N * c1(x)) + (1 - x/N) * c6(x)
    eq6 = c6(x) == (1 - x/N) * (1 + 1/N * KB[c1(x+1)] + (N-x-1) * (1 - (x+1)/N) * (1 + KB[c1(x+2)])) + x/N * (1 - x/N) * (1 + KB[c1(x+1)])
    eqs += [eq5, eq6]
    return (eqs, [cvar(i,x) for i in range(1, 7)])

def change_order_threshold(N, step=1):
    backtrack = []
    for p in range(0, N, step):
        # p positions have fixed values
        c = compute_complexity(
            N, p,
            generate_spritz_variables,
            get_spritz_boundary_conditions,
            lambda N, x, kb: generate_spritz_threshold_equations(N, x, kb, 2*p)
        )
        backtrack.append(c)
        
    ff = falling_factorial
    bruteforce = [ff(N,x) for x in range(0, N, step)]

    complexity = [a*b for a,b in zip(bruteforce,backtrack)]
    # print complexity
    return complexity

In [ ]:
N = 256

In [ ]:
%time co = change_order(N)
with open('co', 'w') as f:
    f.write(str(co))

In [ ]:
%time cot = change_order_threshold(N, step=5)
with open('cot', 'w') as f:
    f.write(str(cot))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(range(0, N, 5), [int(x.log(2)) for x in co[::5]], color='red', label=u'Zafixované pozície')
ax.plot(range(0, N, 5), [int(x.log(2)) for x in cot], color='green', label=u'Zafixované hodnoty z bežiaceho kľúča')

ax.legend(loc=2)
# print ['$2^{'+str(int(i))+'}$' for i in ax.get_yticks().tolist()]
labels = ['$2^{'+str(int(i))+'}$' for i in ax.get_yticks().tolist()]
ax.set_yticklabels(labels)

plt.xlabel(u'Počet prednastavených hodnôt -- $m$', fontsize='larger')
plt.ylabel(u'Celková zložitosť prehľadávania', fontsize='larger')
plt.savefig('tradeoff-256.pdf')